In [1]:
import importlib
cim_profile = 'rc4_2021'
cim = importlib.import_module('cimlab.data_profile.' + cim_profile)


In [2]:
from cimlab.loaders import Parameter, ConnectionParameters
from cimlab.loaders.blazegraph.blazegraph import BlazegraphConnection
from cimlab.loaders.gridappsd import GridappsdConnection, get_topology_response
from cimlab.models.distributed_model import DistributedModel
# import cim.data_profile as cim
import json


In [3]:
# # Blazegraph connection for running inside the container
# params = ConnectionParameters([Parameter(key="url", value="http://blazegraph:8080/bigdata/namespace/kb/sparql")])
# bg = BlazegraphConnection(params)

In [4]:
# Blazegraph connection for running outside the container
params = ConnectionParameters([Parameter(key="url", value="http://localhost:8889/bigdata/namespace/kb/sparql")])
bg = BlazegraphConnection(params, 'rc4_2021')

In [5]:
#feeder_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"  # 123 bus
feeder_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
# feeder_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" # 13 bus asets
#feeder_mrid = "_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3"  # 8500 node
#feeder_mrid = "_67AB291F-DCCD-31B7-B499-338206B9828F" # J1
#feeder_mrid = "_9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095"  # R2 12.47 3
#feeder_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #9500 node

In [6]:
feeder = cim.Feeder(mRID=feeder_mrid)

In [7]:
# from gridappsd import GridAPPSD
from topology_processor import DistributedTopology

gapps = GridappsdConnection(feeder_mrid)
Topology = DistributedTopology(gapps, feeder_mrid)
topology_response = Topology.create_switch_areas(feeder_mrid)
# topology_response = get_topology_response(feeder_mrid)
topology_response = json.loads(topology_response)
# with open('topo_message.json') as topo_file:
#     topology_response = json.load(topo_file)


In [8]:
# topology_response = json.loads(topology_response)

In [9]:
network = DistributedModel(connection=bg, feeder=feeder, topology=topology_response)

In [10]:
switch_area = network.switch_areas[0]

# Get phase, bus info about ACLineSegments
switch_area.get_all_attributes(cim.ACLineSegment)
switch_area.get_all_attributes(cim.ACLineSegmentPhase)
switch_area.get_all_attributes(cim.Terminal)

line_ids = list(switch_area.typed_catalog[cim.ACLineSegment].keys())
for line_id in line_ids:
    print('line mrid: ',line_id)
    print('line name:', switch_area.typed_catalog[cim.ACLineSegment][line_id].name)
    print('bus 1: ', switch_area.typed_catalog[cim.ACLineSegment][line_id].Terminals[0].ConnectivityNode.mRID)
    print('bus 2: ', switch_area.typed_catalog[cim.ACLineSegment][line_id].Terminals[1].ConnectivityNode.mRID)
    for line_phs in switch_area.typed_catalog[cim.ACLineSegment][line_id].ACLineSegmentPhases:
        print('phase ', line_phs.phase, ': ', line_phs.mRID)
    

line mrid:  _0BBD0EA3-F665-465B-86FD-FC8B8466AD53
line name: 645646
bus 1:  _8C58660F-C62C-4903-BE72-22F1255B1E62
bus 2:  _0124E881-B82D-4206-BBDF-37D585159872
phase  C :  _94FCAFFF-E5F7-4F97-B697-35C46656EDCA
phase  B :  _CE2AFB05-A1B1-4082-B4EB-2AD8A1D3422D
line mrid:  _1C6781A2-5B9D-4525-8A9B-F9B32C4C4AC0
line name: 632645
bus 1:  _0124E881-B82D-4206-BBDF-37D585159872
bus 2:  _6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B
phase  C :  _19B6044B-6915-4245-9DD1-6AB0444D42A6
phase  B :  _D9C9CEA6-B3F0-4E02-9911-3DFA192B4D01
line mrid:  _A04CDFB1-E951-4FC4-8882-0323CD70AE3C
line name: 650632
bus 1:  _673E896A-DCBF-4E43-9924-BEB31C5B6005
bus 2:  _6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B
phase  A :  _2A5A1025-AD81-4BFA-A7F0-ED70F7F43DAA
phase  B :  _2D60B241-DE87-48BD-A8D3-FE4B705B40ED
phase  C :  _A66508A9-C9C1-4DB5-8FFD-F081E5E9E14D
line mrid:  _FBE26B35-13AB-457D-9795-DF58B28E309D
line name: 632633
bus 1:  _6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B
bus 2:  _C6256170-E6ED-4F91-8EBD-748090C3FDD5
phase  A :

In [11]:
switch_area = network.switch_areas[0]

switch_area.get_all_attributes(cim.EnergyConsumer)
switch_area.get_all_attributes(cim.EnergyConsumerPhase)

load_ids = list(switch_area.typed_catalog[cim.EnergyConsumer].keys())

for load in load_ids:
    print('p = ', switch_area.typed_catalog[cim.EnergyConsumer][load].p)
    print('q = ', switch_area.typed_catalog[cim.EnergyConsumer][load].q)
    for load_phs in switch_area.typed_catalog[cim.EnergyConsumer][load].EnergyConsumerPhase:
        print('phases: ', load_phs.phase)

p =  230000
q =  132000
phases:  B
p =  170000
q =  125000
phases:  B


In [12]:
network.get_all_attributes(cim.TransformerTank)
network.get_all_attributes(cim.TransformerTankEnd)
network.get_all_attributes(cim.TransformerTankInfo)
network.get_all_attributes(cim.TransformerEndInfo)
network.get_all_attributes(cim.ShortCircuitTest)
network.get_all_attributes(cim.NoLoadTest)

In [13]:
switch_area.get_all_attributes(cim.ACLineSegment)
switch_area.get_all_attributes(cim.ACLineSegmentPhase)
switch_area.get_all_attributes(cim.PerLengthPhaseImpedance)
switch_area.get_all_attributes(cim.PhaseImpedanceData)

switch_area.get_all_attributes(cim.WireSpacingInfo)
switch_area.get_all_attributes(cim.WirePosition)
switch_area.get_all_attributes(cim.OverheadWireInfo)
switch_area.get_all_attributes(cim.ConcentricNeutralCableInfo)
switch_area.get_all_attributes(cim.TapeShieldCableInfo)



In [14]:
switch_area.typed_catalog[cim.ACLineSegment][line_id].PerLengthImpedance.PhaseImpedanceData

[PhaseImpedanceData(mRID='_0BE57FFF-A484-435A-A872-8FE3152AE9D5', aliasName=None, description=None, name=None, Names=[], b='0', column='1', g=None, r='0.000096936556', row='3', x='0.00031175045', PhaseImpedance='_EAEF5A85-027A-4BE2-9A07-99D4550C3DE1'),
 PhaseImpedanceData(mRID='_36CD0283-0C33-45F1-B504-C7EF348D8CC0', aliasName=None, description=None, name=None, Names=[], b='0', column='1', g=None, r='0.00046765675', row='1', x='0.000734108', PhaseImpedance='_EAEF5A85-027A-4BE2-9A07-99D4550C3DE1'),
 PhaseImpedanceData(mRID='_699B3870-6CE0-4B8C-830C-77D071EECD2E', aliasName=None, description=None, name=None, Names=[], b='0', column='2', g=None, r='0.000098179333', row='3', x='0.00026322003', PhaseImpedance='_EAEF5A85-027A-4BE2-9A07-99D4550C3DE1'),
 PhaseImpedanceData(mRID='_7903D890-15F0-4773-814B-0A92B52396AF', aliasName=None, description=None, name=None, Names=[], b='0', column='3', g=None, r='0.00046206425', row='3', x='0.00075262537', PhaseImpedance='_EAEF5A85-027A-4BE2-9A07-99D4550C

In [15]:
print(switch_area.get_attributes_query(cim.PhaseImpedanceData))


        PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX cim:  <http://iec.ch/TC57/CIM100#>
        SELECT ?mRID ?name ?PhaseImpedance ?row ?column ?r ?x ?b 

        WHERE {          
          ?eq r:type cim:PhaseImpedanceData.
          VALUES ?fdrid {"_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"}
          VALUES ?mRID { "_0649C1EF-E39F-4B46-AD80-D3C180E4562E" 
 "_10B0D529-0FC6-4845-BEFE-2CD11D1EC0B4" 
 "_1C9AB18C-CCF4-409A-92A7-865CDA8CC7A5" 
 "_520F9AFD-543F-4AD9-9178-9511763BB59D" 
 "_69B855E7-917B-49C7-B96A-F8C8E3A4CBB5" 
 "_BBC492B3-CAEB-4DF8-B4A3-EAF9D05E7E1E" 
 "_0BE57FFF-A484-435A-A872-8FE3152AE9D5" 
 "_36CD0283-0C33-45F1-B504-C7EF348D8CC0" 
 "_699B3870-6CE0-4B8C-830C-77D071EECD2E" 
 "_7903D890-15F0-4773-814B-0A92B52396AF" 
 "_842054E7-44EB-4010-99B7-475DA622A91D" 
 "_997E83A5-27DC-426D-A88E-56FCFB96146B" 
 "_4BDC7B0A-F286-4355-9902-DEE520D20C27" 
 "_7EF9213B-DA56-4302-9054-C148028273A3" 
 "_C1EF0F7C-9A4B-4524-87C6-7FA30F4ED054" 
               } 
        

In [16]:
json.loads(switch_area.__dumps__(cim.PhaseImpedanceData))

{'_0649C1EF-E39F-4B46-AD80-D3C180E4562E': {'mRID': '_0649C1EF-E39F-4B46-AD80-D3C180E4562E',
  'aliasName': '',
  'description': '',
  'name': '',
  'Names': [],
  'b': '0',
  'column': '1',
  'g': '',
  'r': '0.000095383086',
  'row': '2',
  'x': '0.00023917231',
  'PhaseImpedance': '_50044320-7295-4591-B39E-F793F1923098'},
 '_10B0D529-0FC6-4845-BEFE-2CD11D1EC0B4': {'mRID': '_10B0D529-0FC6-4845-BEFE-2CD11D1EC0B4',
  'aliasName': '',
  'description': '',
  'name': '',
  'Names': [],
  'b': '0',
  'column': '2',
  'g': '',
  'r': '0.000096936556',
  'row': '3',
  'x': '0.00031175045',
  'PhaseImpedance': '_50044320-7295-4591-B39E-F793F1923098'},
 '_1C9AB18C-CCF4-409A-92A7-865CDA8CC7A5': {'mRID': '_1C9AB18C-CCF4-409A-92A7-865CDA8CC7A5',
  'aliasName': '',
  'description': '',
  'name': '',
  'Names': [],
  'b': '0',
  'column': '3',
  'g': '',
  'r': '0.00021214193',
  'row': '3',
  'x': '0.00064301249',
  'PhaseImpedance': '_50044320-7295-4591-B39E-F793F1923098'},
 '_520F9AFD-543F-4AD9-9

In [17]:
secondary_area = network.switch_areas[4].secondary_areas[0]
secondary_area.get_all_attributes(cim.PowerElectronicsConnection)
secondary_area.get_all_attributes(cim.PowerElectronicsConnectionPhase)

pec_ids = list(secondary_area.typed_catalog[cim.PowerElectronicsConnection])

for pec in pec_ids:
    print('p = ', secondary_area.typed_catalog[cim.PowerElectronicsConnection][pec].p)
    print('q = ', secondary_area.typed_catalog[cim.PowerElectronicsConnection][pec].q)
    for pec_phs in secondary_area.typed_catalog[cim.PowerElectronicsConnection][pec].PowerElectronicsConnectionPhases:
        print('phase ', pec_phs.phase, ': ', pec_phs.mRID)

p =  -5000
q =  0
phase  s1 :  _85512A8C-BBA9-414A-B652-5A66FF9511CF
phase  s2 :  _9D5E51F9-1424-4085-AA82-9B4253F8DA16
p =  4000
q =  0
phase  s2 :  _93759898-98F2-4D75-97E3-C57F3F7B20FF
phase  s1 :  _CCB76658-7775-4B97-95C8-9003986F20C3


In [18]:
values = ['_0649C1EF-E39F-4B46-AD80-D3C180E4562E']
message = bg.sparql.get_class_type_sparql(feeder_mrid, values)
output = bg.execute(message)
out = output['results']['bindings']
if values and not out:
    print('yes')

yes
